### Guido's code https://code.earthengine.google.com/73d6637f9a84f4ff2e083a20a74ccbdd 

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()
import time
import datetime
import seaborn as sns

In [41]:
# add dn for product aggregated into 8 days
def add_dn_date(img,beginDate=None,n=None,IncludeYear=False):
    if beginDate is None:
        beginDate = img.get('system:time_start')
    else:
        beginDate = beginDate
    if IncludeYear is False:
        IncludeYear = True
    if n is None:
        n = 8
    beginDate = ee.Date(beginDate)
    year  = beginDate.get('year')
    month = beginDate.get('month')
    diff  = beginDate.difference(ee.Date.fromYMD(year, 1, 1), 'day').add(1)
    dn    = diff.subtract(1).divide(n).floor().add(1).int()
    yearstr  = year.format('%d') 
    dn = dn.format('%02d')
    return ee.Image(img).set('system:time_start', beginDate.millis()).set('date', beginDate.format('yyyy-MM-dd')).set('Year', yearstr).set('Month',beginDate.format('MM')).set('YearMonth', beginDate.format('YYYY-MM')).set('dn', dn)

# wrapper function for add_dn_date
def add_dn_date_all(Year,days):
    def wrapper(image0):
        tmp = add_dn_date(img = image0,IncludeYear=Year,n = days)
        return tmp
    return (wrapper)

# add NDVI to data
def addNDVI(image):
    #image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 

def smooth_func(image): 
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean().rename(['mean']))

def clim5y(month):
    month = ee.String(month)
    seqNDVI = MOD09ndviY.filterMetadata('dn', 'equals',month )
    return seqNDVI.median().copyProperties(seqNDVI.first(), ['system:time_start','system:time_end','dn'])

# filter smoothed map
def filt_smoothed(image):
    return image.updateMask(count_valid.gte(20))

# function for accumulating NDVI
def accumulate(image,list):
    # Get the latest cumulative NDVI of the list with
    # get(-1).  Since the type of the list argument to the function is unknown,
    # it needs to be cast to a List.  Since the return type of get() is unknown,
    # cast it to Image.
    previous = ee.Image(ee.List(list).get(-1)).toFloat()
    # Add the current anomaly to make a new cumulative NDVI image and Propagate metadata to the new image.
    added = image.toFloat().add(previous).toFloat().set('system:time_start', image.get('system:time_start'))
    return ee.List(list).add(added)

# cumulative normalized
def cum_dividelast(image):
    return image.divide(last)

# calculate deviations
def deviations_calc(image):
    tmp = image.select('mean').reproject(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceNeighborhood(reducer='stdDev',kernel= ee.Kernel.square(6, 'pixels'),skipMasked =True)
    return image

In [11]:
# load collections and required images
collection = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2000-01-01', '2020-12-31')
forestmask = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")


In [12]:
# add dn
collection = collection.map(add_dn_date_all(Year = False, days = 8))

In [13]:
# end and start date of period of interest
start_date = ee.Date.fromYMD(2001, 1, 1)
end_date   = ee.Date.fromYMD(2019, 12, 31)

In [14]:
# mask out crap pixels
MOD09masked = collection.filterDate(start_date, end_date).map(maskPixels)

In [15]:
# add NDVI as a new band
MOD09ndvi = MOD09masked.map(addNDVI).select('NDVI')

In [16]:
# list of seasons
tmpseas = ["%02d" % x for x in list(range(1, 46+1))]
tmpseas1 = ee.List(tmpseas)

In [8]:
for year in list(range(2002,2002)):
    # climatology
    yearp5 = ee.Number(year).add(5)
    start_date = ee.Date.fromYMD(year,1,1)
    end_date = ee.Date.fromYMD(yearp5,12,31)
    MOD09ndviY = MOD09ndvi.filterDate(start_date, end_date)
    # climatology 5 years (monthly composites)
    seasons = ee.ImageCollection.fromImages(tmpseas1.map(clim5y))
    # set the width of the temporal smoothing bandwidth (in days)
    bw = 35
    # This field contains UNIX time in milliseconds
    timeField = 'system:time_start'
    # sort by start time
    filteredMODIS = seasons.sort("system:time_start")
    # Smoothing
    join = ee.Join.saveAll(matchesKey = 'images')
    diffFilter = ee.Filter.maxDifference(difference =1000 * 60 * 60 * 24 * bw, 
                                         leftField = timeField,rightField = timeField)
    threeNeighborJoin = join.apply(primary = filteredMODIS,secondary = filteredMODIS,condition = diffFilter)
    # get smoothed collection
    smoothed = ee.ImageCollection(threeNeighborJoin.map(smooth_func))
    # mask smoothed dataset
    #smoothed_masked = smoothed.map(mask_smt)
    # count valid images
    count_valid = smoothed.select('NDVI').count()
    # mask smoothed dataset
    smoothed = smoothed.map(filt_smoothed)
    # select mean band
    smoothed = smoothed.select('mean')
    # Get the timestamp from the most recent image in the reference collection.
    time0 = smoothed.first().get('system:time_start')
    # Rename the first band 'NDVI'.
    first = ee.List([ee.Image(0).set('system:time_start', time0).select([0], ['mean']).toFloat()])
    # Create an ImageCollection of cumulative anomaly images by iterating.
    # Since the return type of iterate is unknown, it needs to be cast to a List.
    cumulative = ee.ImageCollection(ee.List(smoothed.iterate(accumulate, first)))
    # normalise
    last = cumulative.sort('system:time_start', False).first()
    last = last.updateMask(last.gte(9))
    # cumulative map normalised
    cumulativeNorm = cumulative.map(cum_dividelast)
    # deviations (these are described in the document sent by Alessandro)
    cumulativeStd10 = cumulativeNorm.map(deviations_calc)
    # calculate the mean of the deviations
    cumulativeStd10_mean = cumulativeStd10.mean().multiply(10000)
    cumulativeStd10_mean = cumulativeStd10_mean.updateMask(last.gte(9))

In [17]:
# generate palette
pal = sns.color_palette("viridis",20).as_hex()
palette = {'min':4000 , 'max':5800, 'palette':pal}

In [44]:
pkg_smooth = require('users/kongdd/public:Math/pkg_smooth.js')

NameError: name 'require' is not defined

In [42]:
# filter smoothed map
def filt_smoothed(image):
    tmp = image.unmask()
    return tmp.updateMask(count_valid.gte(30))
year = 2002
# climatology
yearp5 = ee.Number(year).add(5)
start_date = ee.Date.fromYMD(year,1,1)
end_date = ee.Date.fromYMD(yearp5,12,31)
MOD09ndviY = MOD09ndvi.filterDate(start_date, end_date)
# climatology 5 years (monthly composites)
seasons = ee.ImageCollection.fromImages(tmpseas1.map(clim5y))
# set the width of the temporal smoothing bandwidth (in days)
bw = 35
# This field contains UNIX time in milliseconds
timeField = 'system:time_start'
# sort by start time
filteredMODIS = seasons.sort("system:time_start")
# Smoothing
join = ee.Join.saveAll(matchesKey = 'images')
diffFilter = ee.Filter.maxDifference(difference =1000 * 60 * 60 * 24 * bw, 
 leftField = timeField,rightField = timeField)
threeNeighborJoin = join.apply(primary = filteredMODIS,secondary = filteredMODIS,condition = diffFilter)
# get smoothed collection
smoothed = ee.ImageCollection(threeNeighborJoin.map(smooth_func))
# mask smoothed dataset
#smoothed_masked = smoothed.map(mask_smt)
# count valid images
count_valid = smoothed.select('mean').count()
# mask smoothed dataset
smoothed = smoothed.map(filt_smoothed)
# select mean band
smoothed = smoothed.select('mean')
# Get the timestamp from the most recent image in the reference collection.
time0 = smoothed.first().get('system:time_start')
# Rename the first band 'NDVI'.
first = ee.List([ee.Image(0).set('system:time_start', time0).select([0], ['mean']).toFloat()])
# Create an ImageCollection of cumulative anomaly images by iterating.
# Since the return type of iterate is unknown, it needs to be cast to a List.
cumulative = ee.ImageCollection(ee.List(smoothed.iterate(accumulate, first)))
# normalise
last = cumulative.sort('system:time_start', False).first()
last = last.updateMask(last.gte(9))
# cumulative map normalised
cumulativeNorm = cumulative.map(cum_dividelast)
# deviations (these are described in the document sent by Alessandro)
cumulativeStd10 = cumulativeNorm.map(deviations_calc)
# calculate the mean of the deviations
cumulativeStd10_mean = cumulativeStd10.mean().multiply(10000)
cumulativeStd10_mean = cumulativeStd10_mean.updateMask(last.gte(9))
#cumulativeStd10_mean1 = cumulativeStd10_mean.updateMask(forestmask.gt(0))
#Map.addLayer(cumulativeStd10_mean1,palette,'metric 6')

In [21]:
#cumulativeStd10_mean1 = cumulativeStd10_mean.updateMask(forestmask.gt(0))
#Map = geemap.Map()

In [19]:

# generate palette
pal = sns.color_palette("coolwarm",20).as_hex()
palette = {'min':4000 , 'max':5800, 'palette':pal}

In [43]:
Map.addLayer(cumulativeStd10_mean,palette,'metric 9')

In [38]:
Map = geemap.Map()

In [40]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…